# Definición de parámetros de simulación (settings)

Comenzamos importando el módulo `openmc`, contiene el API (applicationg programming interface) para generar el input de OpenMC.

In [1]:
import openmc
openmc.config['cross_sections'] = '/home/lucas/Proyecto_Integrador/cross_sections1/endfb80/endfb-viii.0-hdf5/cross_sections.xml'

Una vez que se definieron los materiales y la geometría de nuestra simulación es necesario indicar a OpenMC los parámetros de la simulación. Esto se realiza mediante la función `Settings()`, que genera el archivo `setting.xml` que, con `materials.xml` y `geometry.xml` conforma el conjunto mínimo de archivos de input necesarios para ejecutar OpenMC.

In [2]:
parametros = openmc.Settings()

El tipo de simulación a realizar se establece dandole un valor al parámetro `.run_mode`. Los valores posibles son strings con los siguientes valores:

* `"eigenvalue"`: realiza un cálculo del factor de multiplicación mediante la simulación del reactor crítico asociado.
* `"fixed source"`: realiza un cálculo de fuente fija.
* `"volume"`: realiza un cálculo Monte Carlo del volumen de las celdas de la simulación.
* `"plot"`: genera gráficos 2D y 3D de la geometría.
* `"particle restart"`: permite continuar una simulación.

In [3]:
parametros.run_mode = "eigenvalue"

En los modos `eigenvalue` y `fixed source` debe indicarse el número de partículas a simular subdivididas en grupos denominados *batches*. Esto se especifica con los atributos `.particles` (número de partículas por batch) y `.batches`. Por ejemplo:

In [4]:
parametros.batches = 10
parametros.particles = 1000

realiza una simulación de 10000 partículas, y la simulación se divide en 10 batches de 1000 partículas.

## Simulación de reactor crítico asociado (factor de multiplicación)

Las simulaciones para calcular el factor de multiplicación de un sistema con material físil se realizan en el modo `eigenvalue`. Este modo de ejecución implementa el *método de generaciones sucesivas*. En este método la simulación se subdivide en generaciones (por default, una generación por batch) y en cada generación las partículas son simuladas desde su nacimiento port fisión hasta su desaparición por absorción o fuga. Las partículas secundarias generadas por fisión no son seguidas en forma explícita sino que sus características (posición, energía, dirección) son almacenadas en un banco que sirve como fuente para la próxima generación. La simulación se inicia con una fuente provista por el usuario mediante el atributo `.source`. Durante las primeras generaciones la distribución de fuente va a estar muy alejada de la distribución de equilibrio (la autofunción del sistema), por lo que un número de generaciones inciales debe ser descartado. Esto se indica con el atributo `.inactive_batches`.

### Ejemplo: benchmark Godiva.

Realizaremos la simulación del reactor [Godiva](https://en.wikipedia.org/wiki/Godiva_device). Los detalles de este reactor pueden encontrarse en el benchmark HEU-MET-FAST-001 de [ICSBEP](https://www.oecd-nea.org/science/wpncs/icsbep/) y también en el informe "S. Frankle. A Suite of Criticality Benchmarks for Validating Nuclear Data. LA-13594 (1999)", [disponible en la base de datos INIS de IAEA](https://inis.iaea.org/search/search.aspx?orig_q=RN:32027510).

 <p align="center">
<img src="https://upload.wikimedia.org/wikipedia/commons/2/2a/Godiva-before-scrammed.jpg">
</p>


Godiva es una esfera de $8.7407$ cm de radio de uranio metálico de densidad $18.74$ gm/cm$^3$, $94.73$ % en peso de $^{235}$U, $5.27$ % en peso de $^{238}$U, y $1.02$ % en peso de $^{234}$U.

In [5]:
umet = openmc.Material()
umet.add_nuclide("U235", 0.9473, 'wo')
umet.add_nuclide("U238", 0.0527, 'wo')
umet.add_nuclide("U234", 0.0102, 'wo')
umet.set_density("g/cm3", 18.74)

mats = openmc.Materials([umet])
mats.export_to_xml()

esfera = openmc.Sphere(r=8.74, boundary_type="vacuum")
godiva = openmc.Cell(region = -esfera, fill=umet)
simulacion = openmc.Universe(cells=[godiva])

geom = openmc.Geometry(simulacion)
geom.export_to_xml()

run = openmc.Settings()
run.run_mode = "eigenvalue"
run.particles = 1000
run.batches = 110
run.inactive = 10
run.export_to_xml()

La ejecución del código se realiza con la función `run()` desde el notebook o invocando el comando `openmc` en una terminal abierta en el directorio donde guardamos los archivos xml:

In [6]:
!cat materials.xml

<?xml version='1.0' encoding='utf-8'?>
<materials>
  <material depletable="true" id="1">
    <density units="g/cm3" value="18.74"/>
    <nuclide name="U235" wo="0.9473"/>
    <nuclide name="U238" wo="0.0527"/>
    <nuclide name="U234" wo="0.0102"/>
  </material>
</materials>


In [7]:
!cat geometry.xml

<?xml version='1.0' encoding='UTF-8'?>
<geometry>
  <cell id="1" material="1" region="-1" universe="1"/>
  <surface boundary="vacuum" coeffs="0.0 0.0 0.0 8.74" id="1" type="sphere"/>
</geometry>


In [8]:
!cat settings.xml

<?xml version='1.0' encoding='UTF-8'?>
<settings>
  <run_mode>eigenvalue</run_mode>
  <particles>1000</particles>
  <batches>110</batches>
  <inactive>10</inactive>
</settings>


In [9]:
!rm summary.h5
!rm statepoint.*.h5
openmc.run()

rm: cannot remove 'summary.h5': No such file or directory
rm: cannot remove 'statepoint.*.h5': No such file or directory
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################    

Durante la simulación se lista el valor de $k_\text{eff}$ estimado para cada generación. Las primeras 10 generaciones se descartan, por lo que no aportan al cálculo final del factor de multiplicación final, que se lista en la columna de la derecha. Como se observa la estimación de $k$ comienza con valores muy altos, que se van reduciendo con las generaciones. Esto se debe a que al no espeficar una fuente con el atributo `.source` el default es una fuente isotrópica en el orígen con un espectro de fisión de Watt con parámetros $a= 0.988$ MeV y $b = 2.249$ MeV $^{-1}$. Esto hace que las partículas sean muestreadas en una región del reactor de alta importancia (el centro), lo que genera una sobrestimación incial del factor de multiplicación. Como el reactor es pequeño la fuente converge rápidamente.

Esto lo podemos observar abriendo el archivo `statepoint.110.h5` que (entre otras cosas) contiene el factor de multiplicación para cada generación. Esto puede hacerse con un programa que maneje archivos en formato HDF5 (por ejemplo, `h5dump` del paquete `h5utils`) o utilizando el API en Python de OpenMC. La función `.Statepoint()` abre el archivo y genera un objeto de clase `Statepoint`. La estimación del factor de multiplicación para cada generación puede encontrarse en el atributo `.k_generation`:

In [10]:
estado = openmc.StatePoint("statepoint.110.h5")
keff = estado.k_generation
print(keff)

[1.4043469  1.12342422 1.03980122 1.01815919 0.97057889 0.98140098
 1.00460561 1.06425089 0.97477443 1.0318642  0.97709165 1.02810536
 1.02029819 1.02204093 0.9948815  1.03478568 0.99176716 0.99511476
 1.01076882 1.00553388 0.9791945  0.9901473  1.01551298 1.04539127
 0.95985967 0.94088923 1.01905712 0.98903093 0.97935465 1.01551005
 0.99326321 1.00212711 0.98372335 0.99546617 1.00291157 1.00534817
 1.03228877 0.96189719 0.98444979 1.01358969 1.04395419 0.98663736
 0.99935733 1.05991651 1.00932152 0.99459632 0.98122642 0.97418213
 1.00488255 0.9855936  0.940523   0.99709769 1.00586552 1.0174985
 1.01060725 1.0226227  0.98403609 1.01617791 0.99500175 1.04197574
 1.01900896 0.98048311 1.03182168 0.98701291 0.98768875 0.98943607
 1.01231057 0.96428445 1.02243343 0.97664159 0.96106334 0.99028942
 0.99267614 0.97698259 0.96877007 1.01721274 0.99955826 1.0310255
 1.01269664 1.0231381  0.96772815 0.98097017 0.99784465 0.97478847
 1.02567766 0.97647061 1.04488913 0.99603114 1.0207518  1.032401

In [13]:
%pylab inline
plot(keff)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [14]:
a = hist(keff[11:],bins=10)

La incerteza en el factor de multiplicación es de $200$ pcm ($200\times10^{-5}$). Para reducir la incerteza es necesario aumentar la cantidad de partículas simuladas. Por ejemplo:

In [15]:
run = openmc.Settings()
run.run_mode = "eigenvalue"
run.particles = 10000
run.batches = 110
run.inactive = 10
run.export_to_xml()

In [16]:
!rm summary.h5
!rm statepoint.*.h5
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [17]:
estado2 = openmc.StatePoint("statepoint.110.h5")
keff2 = estado2.k_generation

In [18]:
plot(keff)
plot(keff2)

In [19]:
run = openmc.Settings()
run.run_mode = "eigenvalue"
run.particles = 100000
run.batches = 110
run.inactive = 10
run.export_to_xml()

In [20]:
!rm statepoint.110.h5
!rm summary.h5
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [21]:
estado3 = openmc.StatePoint("statepoint.110.h5")
keff3 = estado3.k_generation

In [22]:
plot(keff)
plot(keff2)
plot(keff3)

In [23]:
kfinal = estado.k_combined
kfinal2 = estado2.k_combined
kfinal3 = estado3.k_combined
print(kfinal)
print(kfinal2)
print(kfinal3)
loglog([100000,1000000,10000000], [kfinal.nominal_value,kfinal2.nominal_value,kfinal3.nominal_value],'.')

1.0018+/-0.0020
0.9999+/-0.0007
1.00075+/-0.00022


/home/lucas/anaconda3/envs/openmc_develop/lib/python3.10/site-packages/openmc/statepoint.py:277: FutureWarning: The 'k_combined' property has been renamed to 'keff' and will be removed in a future version of OpenMC.
  warnings.warn(


El resultado neto es que la incerteza en el factor de multiplicación se reduce en un factor de $1/\sqrt{N}$.

## Simulación de fuente fija

Para realizar un cálculo en modo de fuente externa es necesario definir una fuente. La fuente se especifica en el parámetro `.source` del objeto de clase `Settings`. La fuente es un objeto generado con la función `.IndependentSource()`, que posee como atributos las distribuciones espaciales, angulares y energéticas de la fuente. Por ejemplo, para definir una fuente isotrópica de $1$ MeV en el origen hacemos:

In [24]:
dist_espacial = openmc.stats.Point(xyz=(0.0, 0.0, 0.0))
dist_angular = openmc.stats.Isotropic()
dist_energetica = openmc.stats.Discrete([1.0e6], [1.0])

fuente = openmc.IndependentSource()
fuente.space = dist_espacial
fuente.angle = dist_angular
fuente.energy = dist_energetica

parametros = openmc.Settings()
parametros.source = fuente

Las distribuciones se generan en base al módulo `openmc.stats`. Este módulo posee distintos tipos de distribuciones para las variables energía, posición y dirección, y distribuciones de probabilidad genéricas:

Distribuciones espaciales:

* `openmc.stats.Point()`: distribución delta en un punto del espacio, $\delta(\vec{r}-\vec{r}_0)$.
* `openmc.stats.Box()`: distribución uniformemente distribuida entre un punto (esquina inferior izquierda) y otro (esquina superior derecha).
* `openmc.stats.CartesianIndependent()`: distribución factorizada en distribuciones univariadas para $x$, $y$, $z$.

Distribuciones angulares:
* `openmc.stats.Isotropic()`: distribución isotrópica en la esfera unitaria.
* `openmc.stats.Monodirectional()`: distribución delta en una dirección angular, $\delta(\hat{\Omega}-\hat{\Omega}_0)$.
* `openmc.stats.PolarAzimuthal()`: distribución factorizada en distribuciones univariadas para $\mu = \cos(\theta)$ y $\phi$, respecto a una dirección dada (por defecto, $+z$).

Distribuciones univariadas (que pueden utilizarse también para energía):
* `openmc.stats.Discrete()`: distribución caracterizada por valores discretos con probabilidad asociada.
* `openmc.stats.Uniform()`: distribución con probabilidad constante en un intervalo $[a,b]$.
* `openmc.stats.Tabular()`: distribución continua por trozos con una ley de interpolación (histograma, lin-lin, lin-log, log-lin o log-log`).
* `openmc.stats.Watt()`: distribución de Watt (espectro de fisión: $\chi(E) = C e^{-E/a} \sinh{\sqrt{b E}}$).
* `openmc.stats.Maxwell()`: distribución de Maxwell (distribución térmica: $M(E) = C E e^{-E/kT}$)
* `openmc.stats.Legendre()`: distribución de probabilidad dada por una expansión en polinomios de Legendre.
* `openmc.stats.Mixture()`: distribución discreta en la que los valores son otras distribuciones (similar a `FCEL` en MCNP).

Por ejemplo, la fuga de neutrones de una esfera de agua de radio $5$ cm con una fuente monoenergética :

In [25]:
h2o = openmc.Material()
h2o.add_nuclide("H1", 2.0, "ao")
h2o.add_nuclide("O16", 1.0, "ao")
h2o.set_density("g/cm3", 1)
h2o.add_s_alpha_beta("c_H_in_H2O")

mats = openmc.Materials([h2o])
mats.export_to_xml()

esfera = openmc.Sphere(r=5, boundary_type="vacuum")
celda = openmc.Cell(region=-esfera, fill=h2o)
universo = openmc.Universe(cells=[celda])

geom = openmc.Geometry(universo)
geom.export_to_xml()

dist_espacial = openmc.stats.Point(xyz=(0.0, 0.0, 0.0))
dist_angular = openmc.stats.Isotropic()
dist_energetica = openmc.stats.Discrete([1.0e6], [1.0])

fuente = openmc.IndependentSource()
fuente.space = dist_espacial
fuente.angle = dist_angular
fuente.energy = dist_energetica

parametros = openmc.Settings()
parametros.source = fuente
parametros.run_mode = "fixed source"
parametros.batches = 100
parametros.particles = 1000
parametros.export_to_xml()

In [26]:
!rm statepoint.*.h5
!rm summary.h5
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

## Fuentes distribuidas en cálculos de criticidad

Ahora que sabemos definir fuentes podemos volver a la simulación de Godiva. Como vimos, el estimador del factor de multiplicación estaba sobreestimado en las primeras generaciones porque la fuente inicial estaba concentrada en la región de máxima importancia (el centro de la esfera). Si, por el contrario, ponemos una fuente inicial distribuida en todo el núcleo la aproximación será por abajo:

In [27]:
umet = openmc.Material()
umet.add_nuclide("U235", 0.9473, 'wo')
umet.add_nuclide("U238", 0.0527, 'wo')
umet.add_nuclide("U234", 0.0102, 'wo')
umet.set_density("g/cm3", 18.74)

mats = openmc.Materials([umet])
mats.export_to_xml()

esfera = openmc.Sphere(r=8.74, boundary_type="vacuum")
godiva = openmc.Cell(region = -esfera, fill=umet)
simulacion = openmc.Universe(cells=[godiva])

geom = openmc.Geometry(simulacion)
geom.export_to_xml()

run = openmc.Settings()
run.run_mode = "eigenvalue"
run.particles = 1000
run.batches = 110
run.inactive = 10

fuente = openmc.IndependentSource(constraints={'fissionable': True})
dist_espacial = openmc.stats.Box((-9.0, -9.0, -9.0), (+9.0, +9.0, +9.0))
dist_angular = openmc.stats.Isotropic()
fuente.space = dist_espacial
fuente.angle = dist_angular

run.source = fuente

run.export_to_xml()

In [28]:
!rm statepoint.110.h5
!rm summary.h5
openmc.run()

rm: cannot remove 'statepoint.110.h5': No such file or directory
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 ###################

In [29]:
estado4 = openmc.StatePoint("statepoint.110.h5")
keff4 = estado4.k_generation
plot(keff, label="fuente puntual")
plot(keff4, label="fuente distribuida")
legend()